# Calculando as baselines

In [ ]:
def baseline(indice,semana,folder,folderout,tabela):
    '''
    indice -> indice como escrito na pasta de entrada em string
    semana -> 01 a 20 em string
    folder -> raiz da pasta onde estão os índices de vegetação
    folderout -> onde será salvo
    tabela -> endereço da tabela onde os headers são s01, s02...
              e as datas estão no formato YYYYJJJ
    '''
    
    import glob, gdal, os
    import numpy as np
    import pandas as pd
    import time
    import bottleneck as bn
    
    startTime = time.time()
    #cria pasta de saida
    if not os.path.exists(folderout): os.makedirs(folderout)
        
    #lista de imagens da pasta toda
    lista = []
    for j in glob.glob(folder + '*.tif'):
        b = j.split('/')
        c = b[-1]
        lista.append(c)
        lista.sort()
    
    #adquire metadados da imagem
    datum = gdal.Open(folder + lista[0])
    geo = datum.GetGeoTransform()
    proj = datum.GetProjection()
    col = datum.RasterYSize
    row = datum.RasterXSize
    shape = (row,col)
    driver = gdal.GetDriverByName( 'GTiff' )
    
    
    #cria lista de datas a serem processadas com base na tabela
    datasanos = pd.read_csv(tabela, dtype=str)
    listadatas = datasanos['s'+ semana].tolist() 
    listadatas = [x for x in listadatas if str(x) != 'nan']
    
    #arquivos para calculo da media e mediana semanal
    toOpen = []
    for data in listadatas:
        for imagem in lista:
            if data == imagem[3:10]: #[:-4], [9:16] , [0:7] , [3:10]
                toOpen.append(imagem)
    
    #abre as imagens e monta um pacotão
    arrays = []
    for i in toOpen:
        imagem = gdal.Open(folder + i).ReadAsArray()
        imagem = np.where(imagem < -3 , np.nan, imagem)
        arrays.append(imagem)
        arrays2 = np.dstack(arrays)  
    
    #cálculos 
    media = bn.nanmean(arrays2, axis=2)
    mediana = bn.nanmedian(arrays2, axis=2)
    stdev = np.std(arrays2, axis=2)
    
    #Escrever saidas
    #mediana
    im_saida = driver.Create(folderout + 'mediana' + semana + '.tif', 
                             row, col, 1, gdal.GDT_Float32)
    im_saida.SetGeoTransform(geo)
    im_saida.SetProjection(proj)
    im_saida.GetRasterBand(1).WriteArray(mediana)
    im_saida = None
    
    #media
    im_saida = driver.Create(folderout + 'media' + semana + '.tif', 
                             row, col, 1, gdal.GDT_Float32)
    im_saida.SetGeoTransform(geo)
    im_saida.SetProjection(proj)
    im_saida.GetRasterBand(1).WriteArray(media)
    im_saida = None
    
    #desvio padrao
    im_saida = driver.Create(folderout + 'stdev' + semana + '.tif', 
                             row, col, 1, gdal.GDT_Float32)
    im_saida.SetGeoTransform(geo)
    im_saida.SetProjection(proj)
    im_saida.GetRasterBand(1).WriteArray(stdev)
    im_saida = None
    
    #Finalizando o processo
    mediana, media, imagem, stdev = None, None, None, None
    print 'Processing week ' + semana
    print 'Images saved in ' + folderout
    print 'The process took ', time.time() - startTime, 'seconds'
    print ' '
    return 'FINISHED!'
    

In [ ]:
#indice de vegetacao (como nomeado na pasta)
indice = 'tci' #ndvi evi lswi ndi7 ...
semana = ['01','02','03','04','05','06','07','08',
          '09','10','11','12','13','14','15','16',
          '17','18','19','20']
#/media/denis/GEO/DROUGHT_processed/Vegetation_Indices/RAW/sdi2nd
folder = '/media/denis/GEO/DROUGHT_processed/Vegetation_Indices/RAW/tci/' #FILTRADOS ou RAW
folderout = '/media/denis/GEO/DROUGHT_processed/Vegetation_Indices/baseline/' + indice + '/'
tabela = '/media/denis/GEO/Drought_projetos/TABLES/datas_trabalho.csv'

In [ ]:
for i in semana:
    baseline(indice,i,folder,folderout,tabela)